### Reshaping dataframes

This part of the section shows the different ways in which one can view the same datatable.

* [Crosstab](#useful-crosstab)
* [Merge](#useful-merge)
* [Melt](#useful-melt)
* [Pivot](#useful-pivot)
* [Stack/Unstack](#useful-stack)


In [1]:
import pandas as pd

#### Crosstab <a class="anchor" id="useful-crosstab"></a>

Suppose we have a datatable which has categorical attributes. We wish to see the item counts of various different combinations of categories. Crosstab is the function we would use in such a case.

For example, given below is a dataset of cars with categorical attributes. The categories describe how the price of the car, maintenance, space, etc. The full description can be found here: https://archive.ics.uci.edu/ml/datasets/Car+Evaluation


In [2]:
df_cars = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data', header=None)
df_cars.columns = ['Buying_Price', 'Maintenance', 'Doors', 'Persons', 'Boot_Space', 'Safety', 'Acceptability']

df_cars.sample(5)

,Buying_Price,Maintenance,Doors,Persons,Boot_Space,Safety,Acceptability
659,high,med,2,4,small,high,acc
1219,med,low,3,2,med,med,unacc
133,vhigh,high,2,more,big,med,unacc
1148,med,med,4,4,med,high,vgood
506,high,vhigh,4,more,small,high,unacc


In [3]:
df_cars.shape

(1728, 7)

In [4]:
# We want to see the number of cars of across different price ranges and acceptability.
# Using crosstab, we can read off the numbers quite easily. For example, there are 39 low priced cars
# that are also high acceptable.


pd.crosstab( df_cars['Buying_Price'] , df_cars['Acceptability'], margins=True )

Acceptability,acc,good,unacc,vgood,All
Buying_Price,,,,,
high,108,0,324,0,432
low,89,46,258,39,432
med,115,23,268,26,432
vhigh,72,0,360,0,432
All,384,69,1210,65,1728


#### Merge (similar to join in SQL) <a class="anchor" id="useful-merge"></a>

In [5]:
import pandas as pd

df_bikes = pd.read_csv('misc/bike_price.csv')
df_type= pd.read_csv('misc/bike_type.csv')

In [6]:
pd.merge(df_bikes,df_type,on='TypeNumber',how='inner')

,Model,Price (Rs),TypeNumber,Type
0,Hero Ranger DTB,7000,1,Mountain Bike
1,BSA Photon,8499,1,Mountain Bike
2,Bergamont Roxter,20000,1,Mountain Bike
3,Bergamont Vitox,19000,1,Mountain Bike
4,BSA Photon,7499,2,Road Bike
5,Hero Flash,2184,2,Road Bike
6,Cervelo R3 Ultegra,394000,2,Road Bike
7,Ridley Helium X,237100,2,Road Bike
8,Hero Ranger,6299,3,City Bike
9,BSA Photon,9499,3,City Bike


#### Melt <a class="anchor" id="useful-melt"></a>

Melt is a function that reshapes the dataframe by converting column names into values.

In [7]:
temp_week = {
    'Channel': [ 'BT-TV' ,'CNN','BBC', 'Google'],
    'Mon': [26,26,27,25],
    'Tue': [25,26,27,25],
    'Wed': [27,26,27,25],
    'Thu': [29,28,28,28],
    'Fri': [26,26,27,26],
    'Sat': [26,24,27,25],
    'Sun': [23,23,23,22]
       
}
df = pd.DataFrame(data=temp_week)
df = df[ ['Channel','Mon','Tue','Wed','Thu','Fri','Sat','Sun'] ] # cols by default are sored in alphabetical order
df

,Channel,Mon,Tue,Wed,Thu,Fri,Sat,Sun
0,BT-TV,26,25,27,29,26,26,23
1,CNN,26,26,26,28,26,24,23
2,BBC,27,27,27,28,27,27,23
3,Google,25,25,25,28,26,25,22


In [8]:
df.set_index(keys='Channel')

,Mon,Tue,Wed,Thu,Fri,Sat,Sun
Channel,,,,,,,
BT-TV,26,25,27,29,26,26,23
CNN,26,26,26,28,26,24,23
BBC,27,27,27,28,27,27,23
Google,25,25,25,28,26,25,22


In [9]:
temp_df = pd.melt(df, id_vars=['Channel'], var_name='Day', value_name='Temperature')
temp_df

,Channel,Day,Temperature
0,BT-TV,Mon,26
1,CNN,Mon,26
2,BBC,Mon,27
3,Google,Mon,25
4,BT-TV,Tue,25
5,CNN,Tue,26
6,BBC,Tue,27
7,Google,Tue,25
8,BT-TV,Wed,27
9,CNN,Wed,26


#### Pivot  <a class="anchor" id="useful-pivot"></a>

Pivot is the reverse of melt.

In [10]:
temp_df.pivot(index='Channel', columns='Day', values='Temperature')

Day,Fri,Mon,Sat,Sun,Thu,Tue,Wed
Channel,,,,,,,
BBC,27,27,27,23,28,27,27
BT-TV,26,26,26,23,29,25,27
CNN,26,26,24,23,28,26,26
Google,26,25,25,22,28,25,25


In [11]:
# Pivot_table can be used to get aggregate measures. But the prefered way to do this is groupby
import numpy as np
pd.pivot_table(temp_df, columns=['Day'], values='Temperature', aggfunc=np.mean)

Day,Fri,Mon,Sat,Sun,Thu,Tue,Wed
Temperature,26.25,26.0,25.5,22.75,28.25,25.75,26.25


#### Stack/Unstack <a class="anchor" id="useful-stack"></a>

These functions are similar to pivot and melt except that they work on multi-level indexed tables. Let us suppose we have a dataframe with multi-level index as shown below. We can convert the typenumber into a column similar sing unstack. 

In [12]:
df_bikes.sort_values(by=['TypeNumber'],inplace=True)
df_multi = df_bikes.set_index(['TypeNumber','Model'])
df_multi

Price (Rs)
TypeNumber Model                         
1          Hero Ranger DTB           7000
           BSA Photon                8499
           Bergamont Roxter         20000
           Bergamont Vitox          19000
2          BSA Photon                7499
           Hero Flash                2184
           Cervelo R3 Ultegra      394000
           Ridley Helium X         237100
3          Hero Ranger               6299
           BSA Photon                9499
           Specialized Sirrus       37000
           Cannondale Quick         27000

In [13]:
df_unstacked = df_multi.unstack(level='TypeNumber')
df_unstacked

Price (Rs)                   
TypeNumber                  1         2        3
Model                                           
BSA Photon             8499.0    7499.0   9499.0
Bergamont Roxter      20000.0       NaN      NaN
Bergamont Vitox       19000.0       NaN      NaN
Cannondale Quick          NaN       NaN  27000.0
Cervelo R3 Ultegra        NaN  394000.0      NaN
Hero Flash                NaN    2184.0      NaN
Hero Ranger               NaN       NaN   6299.0
Hero Ranger DTB        7000.0       NaN      NaN
Ridley Helium X           NaN  237100.0      NaN
Specialized Sirrus        NaN       NaN  37000.0

In [14]:
# Once would expect stack to do undo the unstack operation, but it does not quite do so. 
# They reason is because stack() has a preference for what is attribute is picked as the stacked level.
# Understanding the exact behaviour involves some info about multi-indexes which we have not discussed.

df_unstacked.stack('TypeNumber')

Price (Rs)
Model              TypeNumber            
BSA Photon         1               8499.0
                   2               7499.0
                   3               9499.0
Bergamont Roxter   1              20000.0
Bergamont Vitox    1              19000.0
Cannondale Quick   3              27000.0
Cervelo R3 Ultegra 2             394000.0
Hero Flash         2               2184.0
Hero Ranger        3               6299.0
Hero Ranger DTB    1               7000.0
Ridley Helium X    2             237100.0
Specialized Sirrus 3              37000.0